In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from torch.utils.data import Dataset, DataLoader

In [ ]:
class BasicDataset(Dataset):
    def __init__(self):
        print("init dataset")
    
    @property
    def n_users(self):
        raise NotImplementedError
    
    @property
    def m_items(self):
        raise NotImplementedError
    
    @property
    def trainDataSize(self):
        raise NotImplementedError
    
    @property
    def testDict(self):
        raise NotImplementedError
    
    @property
    def allPos(self):
        raise NotImplementedError
    
    def getUserItemFeedback(self, users, items):
        raise NotImplementedError
    
    def getUserPosItems(self, users):
        raise NotImplementedError
    
    def getUserNegItems(self, users):
        """
        not necessary for large dataset
        it's stupid to return all neg items in super large dataset
        """
        raise NotImplementedError
    
    def getSparseGraph(self):
        """
        build a graph in torch.sparse.IntTensor.
        Details in NGCF's matrix form
        A = 
            |I,   R|
            |R^T, I|
        """
        raise NotImplementedError

In [ ]:
import csv
csv_file = input('ml100k/movies.csv')
txt_file = input('ml100k/movies_list.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [ ]:
import csv
csv_file = input('ml100k/ratings.csv')
txt_file = input('ml100k/train.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [ ]:
txt_file = input('ml100k/user_list.txt')
with open(txt_file, "w") as my_output_file:
    for i in range(609):
        
        [ my_output_file.write("user_"+ str(i+1) +" "+ str(i+1) +'\n')]
    my_output_file.close()

In [3]:
train_file = '/Users/sayamsingla/Desktop/Warwick/LightGCN-Movie/LightGCN_MovieLens/data/gowalla/train.txt'
train_file2 = '/Users/sayamsingla/Desktop/Warwick/LightGCN-Movie/LightGCN_MovieLens/data/ml100k/train.txt'
train_file3 = '/Users/sayamsingla/Desktop/Warwick/LightGCN-Movie/LightGCN_MovieLens/data/ml100k/map.txt'
path = '/Users/sayamsingla/Desktop/Warwick/LightGCN-Movie/LightGCN_MovieLens/code/ml100k'

In [17]:
trainUniqueUsers, trainItem, trainUser , trainRating= [], [], [], []
testUniqueUsers, testItem, testUser = [], [], []
trainDataSize = 0
testDataSize = 0
n_user = 0
m_item = 0
ref = 0
c = []
map = [0]*193610
a = 1
with open(train_file3) as f:
    
            for l in f.readlines():
                
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    #items = [int(i) for i in l[1]]
                    map_items = int(l[1])
                    
                    if map[map_items] == 0:
                        map[map_items] = a
                        a+=1
                    

with open(train_file2) as f:
            for l in f.readlines():
                
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    #items = [int(i) for i in l[1]]
                    items = int(l[1])
                    rating = float(str(l[2])) 
                    
                    
                    uid = int(l[0])
                    
                    if uid > ref:
                        trainUniqueUsers.append(uid)
                        ref = uid
                    
                    trainItem.append(map[items])
                    trainRating.append(rating)
                    trainUser.append(uid)
                    #trainItem.extend(items)
                    #trainItem.append(items)
                    m_item = max(m_item, items)
                    n_user = max(n_user, uid)
                    trainDataSize += 1
c = trainItem
m_item = len(np.array(list(set(c))))

trainUniqueUsers = np.array(trainUniqueUsers)
trainUser = np.array(trainUser)
trainItem = np.array(trainItem)
trainRating = np.array(trainRating)
m_item = max(trainItem)

In [18]:
size = trainItem.max()
trainUser, trainItem, m_item 

(array([  1,   1,   1, ..., 610, 610, 610]),
 array([   1,    3,    6, ..., 9445, 9446, 9486]),
 9724)

In [19]:
trainItem.shape , trainUser.shape , trainRating.shape , m_item, n_user , trainDataSize

((100836,), (100836,), (100836,), 9724, 610, 100836)

In [7]:
trainUniqueUsers, trainItem, trainUser = [], [], []
testUniqueUsers, testItem, testUser = [], [], []
trainDataSize = 0
testDataSize = 0
n_user = 0
m_item = 0
c =   0
with open(train_file) as f:
            for l in f.readlines():
                c +=1
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    trainUniqueUsers.append(uid)
                    trainUser.extend([uid] * len(items))
                    trainItem.extend(items)
                    m_item = max(m_item, max(items))
                    n_user = max(n_user, uid)
                    trainDataSize += len(items)
                    
trainUniqueUsers = np.array(trainUniqueUsers)
trainUser = np.array(trainUser)
trainItem = np.array(trainItem)
c

29858

In [8]:
    trainUser, trainItem, m_item
    arr = trainItem
    res = 1
 
   

In [9]:
trainItem.shape, len(trainUser), res

((810128,), 810128, 1)

In [20]:
def getUserPosItems( users):
        posItems = []
        for user in users:
            posItems.append(UserItemNet[user].nonzero()[1])
        return posItems

In [21]:
        Graph = None
        print(f"{trainDataSize} interactions for training")
        print(f"{testDataSize} interactions for testing")
        print(f"Sparsity : {(trainDataSize + testDataSize) / n_user / m_item}")
        m_item += 1
        n_user += 1
        #print(n_user, size)
        # (users,items), bipartite graph
        UserItemNet = csr_matrix((np.ones(len(trainUser)), (trainUser, trainItem)),shape=(n_user, m_item))
#         UserItemNet = csr_matrix((trainRating, (trainUser, trainItem)),shape=(n_user, m_item))
        print(UserItemNet)
        
        users_D = np.array(UserItemNet.sum(axis=1)).squeeze()
        users_D[users_D == 0.] = 1
        items_D = np.array(UserItemNet.sum(axis=0)).squeeze()
        items_D[items_D == 0.] = 1.
        # pre-calculate
        _allPos = getUserPosItems(list(range(n_user)))
        #__testDict = __build_test()
        len(_allPos)

100836 interactions for training
0 interactions for testing
Sparsity : 0.016999683055613623
  (1, 1)	1.0
  (1, 3)	1.0
  (1, 6)	1.0
  (1, 44)	1.0
  (1, 47)	1.0
  (1, 63)	1.0
  (1, 90)	1.0
  (1, 98)	1.0
  (1, 125)	1.0
  (1, 131)	1.0
  (1, 137)	1.0
  (1, 185)	1.0
  (1, 191)	1.0
  (1, 198)	1.0
  (1, 202)	1.0
  (1, 225)	1.0
  (1, 258)	1.0
  (1, 276)	1.0
  (1, 292)	1.0
  (1, 308)	1.0
  (1, 315)	1.0
  (1, 321)	1.0
  (1, 326)	1.0
  (1, 368)	1.0
  (1, 385)	1.0
  :	:
  (610, 9239)	1.0
  (610, 9247)	1.0
  (610, 9257)	1.0
  (610, 9269)	1.0
  (610, 9275)	1.0
  (610, 9280)	1.0
  (610, 9283)	1.0
  (610, 9289)	1.0
  (610, 9305)	1.0
  (610, 9308)	1.0
  (610, 9313)	1.0
  (610, 9318)	1.0
  (610, 9325)	1.0
  (610, 9340)	1.0
  (610, 9342)	1.0
  (610, 9349)	1.0
  (610, 9372)	1.0
  (610, 9373)	1.0
  (610, 9375)	1.0
  (610, 9416)	1.0
  (610, 9417)	1.0
  (610, 9444)	1.0
  (610, 9445)	1.0
  (610, 9446)	1.0
  (610, 9486)	1.0


611

In [ ]:
n_users = n_user
m_items = m_item


In [ ]:
        if Graph is None:
            try:
                pre_adj_mat = sp.load_npz(path + '/s_pre_adj_mat.npz')
                print("successfully loaded...")
                norm_adj = pre_adj_mat
            except :
                print("generating adjacency matrix")
                #s = time()
                adj_mat = sp.dok_matrix((n_users + m_items, n_users + m_items), dtype=np.float32)
                adj_mat = adj_mat.tolil()
                R = UserItemNet.tolil()
                adj_mat[:n_users, n_users:] = R
                adj_mat[n_users:, :n_users] = R.T
                adj_mat = adj_mat.todok()
                # adj_mat = adj_mat + sp.eye(adj_mat.shape[0])
                
                rowsum = np.array(adj_mat.sum(axis=1))
                d_inv = np.power(rowsum, -0.5).flatten()
                d_inv[np.isinf(d_inv)] = 0.
                d_mat = sp.diags(d_inv)
                
                norm_adj = d_mat.dot(adj_mat)
                norm_adj = norm_adj.dot(d_mat)
                norm_adj = norm_adj.tocsr()
                #end = time()
                print(f"costing s, saved norm_mat...")
                sp.save_npz(path + '/s_pre_adj_mat.npz', norm_adj)

            if self.split == True:
                self.Graph = _split_A_hat(norm_adj)
                print("done split matrix")
            else:
                self.Graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
                self.Graph = self.Graph.coalesce().to(world.device)
                print("don't split the matrix")

In [ ]:
class Loader(BasicDataset):
    """
    Dataset type for pytorch \n
    Incldue graph information
    gowalla dataset
    """

    def __init__(self,path="../data/gowalla"):
        # train or test
       
        self.mode_dict = {'train': 0, "test": 1}
        self.mode = self.mode_dict['train']
        self.n_user = 0
        self.m_item = 0
        train_file = path + '/train.txt'
        test_file = path + '/test.txt'
        self.path = path
        trainUniqueUsers, trainItem, trainUser = [], [], []
        testUniqueUsers, testItem, testUser = [], [], []
        self.traindataSize = 0
        self.testDataSize = 0

        with open(train_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    trainUniqueUsers.append(uid)
                    trainUser.extend([uid] * len(items))
                    trainItem.extend(items)
                    self.m_item = max(self.m_item, max(items))
                    self.n_user = max(self.n_user, uid)
                    self.traindataSize += len(items)
        self.trainUniqueUsers = np.array(trainUniqueUsers)
        self.trainUser = np.array(trainUser)
        self.trainItem = np.array(trainItem)

        with open(test_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    testUniqueUsers.append(uid)
                    testUser.extend([uid] * len(items))
                    testItem.extend(items)
                    self.m_item = max(self.m_item, max(items))
                    self.n_user = max(self.n_user, uid)
                    self.testDataSize += len(items)
        self.m_item += 1
        self.n_user += 1
        self.testUniqueUsers = np.array(testUniqueUsers)
        self.testUser = np.array(testUser)
        self.testItem = np.array(testItem)
        
        self.Graph = None
        print(f"{self.trainDataSize} interactions for training")
        print(f"{self.testDataSize} interactions for testing")
        print(f"Sparsity : {(self.trainDataSize + self.testDataSize) / self.n_users / self.m_items}")

        # (users,items), bipartite graph
        self.UserItemNet = csr_matrix((np.ones(len(self.trainUser)), (self.trainUser, self.trainItem)),
                                      shape=(self.n_user, self.m_item))
        print(self.n_user, self.m_item)
        self.users_D = np.array(self.UserItemNet.sum(axis=1)).squeeze()
        self.users_D[self.users_D == 0.] = 1
        self.items_D = np.array(self.UserItemNet.sum(axis=0)).squeeze()
        self.items_D[self.items_D == 0.] = 1.
        # pre-calculate
        self._allPos = self.getUserPosItems(list(range(self.n_user)))
        self.__testDict = self.__build_test()
        

    @property
    def n_users(self):
        return self.n_user
    
    @property
    def m_items(self):
        return self.m_item
    
    @property
    def trainDataSize(self):
        return self.traindataSize
    
    @property
    def testDict(self):
        return self.__testDict

    @property
    def allPos(self):
        return self._allPos

    def _split_A_hat(self,A):
        A_fold = []
        fold_len = (self.n_users + self.m_items) // self.folds
        for i_fold in range(self.folds):
            start = i_fold*fold_len
            if i_fold == self.folds - 1:
                end = self.n_users + self.m_items
            else:
                end = (i_fold + 1) * fold_len
            A_fold.append(self._convert_sp_mat_to_sp_tensor(A[start:end]).coalesce().to(world.device))
        return A_fold

    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo().astype(np.float32)
        row = torch.Tensor(coo.row).long()
        col = torch.Tensor(coo.col).long()
        index = torch.stack([row, col])
        data = torch.FloatTensor(coo.data)
        return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))
        
    def getSparseGraph(self):
        print("loading adjacency matrix")
        if self.Graph is None:
            try:
                pre_adj_mat = sp.load_npz(self.path + '/s_pre_adj_mat.npz')
                print("successfully loaded...")
                norm_adj = pre_adj_mat
            except :
                print("generating adjacency matrix")
                s = time()
                adj_mat = sp.dok_matrix((self.n_users + self.m_items, self.n_users + self.m_items), dtype=np.float32)
                adj_mat = adj_mat.tolil()
                R = self.UserItemNet.tolil()
                adj_mat[:self.n_users, self.n_users:] = R
                adj_mat[self.n_users:, :self.n_users] = R.T
                adj_mat = adj_mat.todok()
                # adj_mat = adj_mat + sp.eye(adj_mat.shape[0])
                
                rowsum = np.array(adj_mat.sum(axis=1))
                d_inv = np.power(rowsum, -0.5).flatten()
                d_inv[np.isinf(d_inv)] = 0.
                d_mat = sp.diags(d_inv)
                
                norm_adj = d_mat.dot(adj_mat)
                norm_adj = norm_adj.dot(d_mat)
                norm_adj = norm_adj.tocsr()
                end = time()
                print(f"costing {end-s}s, saved norm_mat...")
                sp.save_npz(self.path + '/s_pre_adj_mat.npz', norm_adj)

            if self.split == True:
                self.Graph = self._split_A_hat(norm_adj)
                print("done split matrix")
            else:
                self.Graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
                self.Graph = self.Graph.coalesce().to(world.device)
                print("don't split the matrix")
        return self.Graph

    def __build_test(self):
        """
        return:
            dict: {user: [items]}
        """
        test_data = {}
        for i, item in enumerate(self.testItem):
            user = self.testUser[i]
            if test_data.get(user):
                test_data[user].append(item)
            else:
                test_data[user] = [item]
        return test_data

    def getUserItemFeedback(self, users, items):
        """
        users:
            shape [-1]
        items:
            shape [-1]
        return:
            feedback [-1]
        """
        # print(self.UserItemNet[users, items])
        return np.array(self.UserItemNet[users, items]).astype('uint8').reshape((-1,))

    def getUserPosItems(self, users):
        posItems = []
        for user in users:
            posItems.append(self.UserItemNet[user].nonzero()[1])
        return posItems

    # def getUserNegItems(self, users):
    #     negItems = []
    #     for user in users:
    #         negItems.append(self.allNeg[user])
    #     return negItems

In [ ]:
 l = Loader()